In [98]:
import numpy as np
import math

# a pure numeric format for the problem data
# example is from github/odwan
# news-vendor problem

# agent buy x units
coefficient_first_stage = [-1]
# price for agent
bound_first_stage = [0,1000]
# units the agent can buy

# agent sell u units
coefficient_second_stage = [1.5]
bound_second_stage = [0,1000]
# it is subject to undeterministic quantaty
# u<=d
# u<=x

# probability distribution for the demand d
# (d,probability)
scenario_second_stage = [(10,0.1),(14,0.4),(16,0.3),(18,0.2)]

# since the optimal x is on the interval (10,18) due to the simplicity
# alternatively, blender decomposition can solve problem too
# however, fit in all possible x can give a good snapshot of the benchmark

benchmarks = [ [ (0.5*min(x,d)-max(x-d,0),prob) for d,prob in scenario_second_stage ] for x in range(10,19)] 
# for sake convinence:
variables = { 'first_stage':['x'],
             'second_stage':['u']
           }
coefficients = {'x':-1,'u':1.5}
random_variables = ['d']

In [176]:
x = 14
def second_stage_rewards(x,scenario_second_stage):
    return [ (1.5*min(x,d),prob) for d,prob in scenario_second_stage ] 
second_stage_rewards(x, scenario_second_stage)

[(15.0, 0.1), (21.0, 0.4), (21.0, 0.3), (21.0, 0.2)]

In [177]:
obj_val = [ (b,sum(x[0]*x[1] for x in benchmarks[b-10])) for b in range(10,19) ]
obj_val

[(10, 5.0),
 (11, 5.35),
 (12, 5.7),
 (13, 6.05),
 (14, 6.4),
 (15, 6.15),
 (16, 5.9),
 (17, 5.2),
 (18, 4.5)]

In [48]:
# the fifth one (x=14) is the optimal in non-SSD
benchmarks[4]

[(1.0, 0.1), (7.0, 0.4), (7.0, 0.3), (7.0, 0.2)]

In [181]:
tem = []
for b in benchmarks:
    tem.extend([v for v,_ in b]) 
all_v = list(set(tem))
all_v.sort()
#print(all_v)
breakpoints = np.array([ [sum(max(v-v_,0)*p for v_,p in b) for v in all_v] for b in benchmarks])
breakpoints[5] - breakpoints[4]
# from the comparison, x= 15 dominates x=14 by SSD

array([0.  , 0.  , 0.  , 0.  , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ,
       0.3 , 0.5 , 0.25, 0.25, 0.25, 0.25])

In [193]:
# from the above exploration results
# we can set one benchmark for our ssd problem as x=14
x = 15

def ssd_benchmark_by_x(x, scenario_second_stage):
    r = second_stage_rewards(x, scenario_second_stage)
    return {
    'first_stage':{'r':[-x], 's':[1.0] },
    'second_stage':{'r':[v for v,p in r], 's':[p for v,p in r]}
}
print(ssd_benchmark_by_x(15,scenario_second_stage))

{'first_stage': {'r': [-15], 's': [1.0]}, 'second_stage': {'r': [15.0, 21.0, 22.5, 22.5], 's': [0.1, 0.4, 0.3, 0.2]}}


In [95]:
# meta parameters
max_itr = 100

In [197]:
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict

def two_stage_ssd_solver(var, coef, r_v, prob_dist, ssd_benchmark):
    itr =0
    # first stage
    m = gp.Model('first_stage')
    x = m.addVar(vtype = GRB.CONTINUOUS, name = 'x')
    # z = m.addVar(vtype = GRB.CONTINUOUS, name = 'z')
    sigma = m.addVar(vtype = GRB.CONTINUOUS, name = 'sigma')
    obj = m.addVar(vtype = GRB.CONTINUOUS, name = 'obj_v')
    # constraint associated with z
    # m.addConstr(z+x<=0)
    
    m.addConstr(sigma == -x -ssd_benchmark['first_stage']['r'][0] )
    
    
    # constriants associated with SSD
    # need to calculate the short-fall w 
    y = defaultdict(float)
    for r,prob in zip(ssd_benchmark['second_stage']['r'], ssd_benchmark['second_stage']['s']):
        y[r]+=prob
    w = {}
    for y_j in y.keys():
        w[y_j] = sum( max(y_j - y_i,0)*prob for y_i,prob in y.items())
    
    
    x_d_aug = {}

    for d,p in prob_dist:
        tem_x_d_aug = m.addVar(vtype = GRB.CONTINUOUS, name = f"min(x,{d})")
        # an auxiliary variable for min(x,d)
        x_d_aug[d] = tem_x_d_aug
        # store in the dict 

        m.addConstr(tem_x_d_aug<=x)
        m.addConstr(tem_x_d_aug<=d)
        
    aug_f = defaultdict(dict)
    for y_j,w_j in w.items():
        tot = 0
        for d,p in prob_dist:
            tem_aug_f = m.addVar(vtype = GRB.CONTINUOUS, name = f'({y_j}-sigma - Q^{d}_2(x))_+')
            # an auxiliary variable for (y_j-sigma - Q^d_i_2(x))_+
            # f_X(d_i,x) in this case is 1.5*min(x,d_i) which is 1.5 x_d_aug[d_i]
            aug_f[d][y_j] = tem_aug_f
            m.addConstr( y_j-sigma - 1.5*x_d_aug[d]<=tem_aug_f)
            m.addConstr(tem_aug_f>= 0)
            tot+=p*tem_aug_f
        m.addConstr( tot <= w_j)
    
    #   problem.setObjective(z+sum( 1.5*x_d_aug[d]*p+ x_d_aug[d] -x_aug[d] for d,p in prob_dist ), GRB.MAXIMIZE)
    m.addConstr(obj == -x+sum( 1.5*x_d_aug[d]*p for d,p in prob_dist))
    m.setObjective(-x+sum( 1.5*x_d_aug[d]*p-sum(tem for _,tem in aug_f[d].items() ) for d,p in prob_dist) , GRB.MAXIMIZE)
    m.optimize()
    # print('DIS')
    # print (m.display())
    x_ = x.X
    z_ = -x.X
    sigma_ = sigma.X
    obj_ = obj.X
    
    
#     while(itr<max_itr):
        
        
#         rewards = [ 1.5*min(x_,d) for d,_ in prob_dist]
#         new_events = []
#         for j,(y_j, w_j) in enumerate(zip(self.y_3,self.w)): # this for can be parallelism
#             if self.p*((y_j - s)*np.ones(self.n) - rewards) <= w_j:
#                 continue
#             event = [ i for i in range(L) if y_j -s > rewards[i]] 
#             new_events.append(event)
#         if not new_events:
#             break
#         else:
#             # add new event cuts as constr
            
#             problem.optimize()
#             # update x,z,sigma
#             x_ = x.X
#             z_ = z.X
#             sigma_ = sigma.X
#             itr+=1
    return x_,z_,sigma_,obj_
two_stage_ssd_solver(variables, coefficients, random_variables, scenario_second_stage, ssd_benchmark)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 37 rows, 19 columns and 80 nonzeros
Model fingerprint: 0x9ca142b6
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 2e+01]
Presolve removed 18 rows and 6 columns
Presolve time: 0.01s
Presolved: 19 rows, 13 columns, 53 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4404800e+01   1.144150e+01   0.000000e+00      0s
       8   -6.4928571e+00   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective -6.492857143e+00


(13.285714285714278,
 -13.285714285714278,
 1.7142857142857224,
 6.149999999999999)

In [198]:
two_stage_ssd_solver(variables, coefficients, random_variables, scenario_second_stage, ssd_benchmark_by_x(16,scenario_second_stage))

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 37 rows, 19 columns and 80 nonzeros
Model fingerprint: 0x42c0c915
Coefficient statistics:
  Matrix range     [1e-01, 2e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 2e+01]
Presolve removed 18 rows and 6 columns
Presolve time: 0.01s
Presolved: 19 rows, 13 columns, 53 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4705100e+01   6.627125e+00   0.000000e+00      0s
       8   -7.3857143e+00   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective -7.385714286e+00


(12.571428571428575,
 -12.571428571428575,
 3.4285714285714235,
 5.900000000000003)